# 00. Baseline - Análisis del Modelo Físico Actual sin Uso de Modelos de Aprendizaje

Hasta ahora, la fragilización se modela con la ecuación ASTM e9000 15:

$$ TTS = TTS_1 + TTS_2 $$

siendo $TTS_1$

$$ TTS_1 = A\cdot\frac{5}{9}\cdot1.8943\cdot10^{-12} \cdot \Phi^{0.5695} 
\left( \frac{1.8\cdot T+32}{550} \right)^{-5.47} $$
$$ \left( 0.09 + \frac{P}{0.012} \right)^{0.216}\cdot\left( 1.66+\frac{Ni^{8.54}}{0.63} \right)^{0.39} \cdot \left( \frac{Mn}{1.36} \right)^{0.3} $$

y siendo $TTS_2$

$$ TS_2 = \frac{5}{9} \cdot \max \left[ \min (Cu, 0.28) - 0.053, 0 \right] \cdot M $$

con $M$

$$ M = B \cdot \max \left\{ \min \left[ 113.87 \left( \ln (\Phi) - \ln(4.5 \times 10^{20}) \right), 612.6 \right], 0 \right\} $$
$$ \cdot \left( \frac{1.8 \cdot T + 32}{550} \right)^{-5.45} \left( 0.1 + \frac{P}{0.012} \right)^{-0.098} \left( 0.168 + \frac{Ni^{0.58}}{0.63} \right)^{0.73} $$

donde $A$ y $B$ son dos constantes que dependen de la categoría ```product_form```; $Cu$, $Ni$, $Mn$ y $P$ son porcentajes matemáticos, la temperatura está en ºC, la fluencia en $n/\mathrm{m^2}$ y la $TTS$ tiene unidades de ºC.

En esa baseline, se pueden aplicar métricas de evaluación para comprobar que tan efectiva es sobre los datos. Para evaluar este proyecto se utilizará el $RMSE$, que se basa en como se alejan las predicciones del modelo de los valores reales observados. La razón de uso de esta métrica es que se expresa en las mismas unidades que la variable de estudio, lo que facilita su interpretación. Con la efectividad del modelo obtenida, se tiene ya una meta cuantitativa a superar.

Se procede al cálculo del $RMSE$, para ello, primero se importan los módulos.

In [7]:
# Primeramente se importan librerias generales 
import numpy as np

# Y luego se importan también las librerías necesarias creadas dentro del proyecto
from sklearn.metrics import mean_squared_error
from src.astm.formula import astm_e900_15
from src.processing.dataset import cargar_dataset


Ahora se define una función que prediga, a partir del dataset, los valores esperados usando el modelo previamente generado en el paquete ```src```; y que calcule el $RMSE$. Después se ejecuta la función.

In [11]:
def calcular_rmse_sin_ai():
    df =cargar_dataset()
    

    try:
        preds_fisica = astm_e900_15(
            cu=df['Cu'].values,
            ni=df['Ni'].values,
            p=df['P'].values,
            mn=df['Mn'].values,
            temp_c=df['Temperature_Celsius'].values,
            fluence=df['Fluence_n_cm2'].values,
            product_form=df['Product_Form'].values
        )
        
        # Guardamos la predicción en el dataframe por si queremos verla luego
        df['Pred_ASTM'] = preds_fisica
        
        # 3. CALCULAR ERROR (RMSE)
        rmse = np.sqrt(mean_squared_error(df['DT41J_Celsius'], df['Pred_ASTM']))
        
        print("-" * 16)
        print(f"RMSE: {rmse:.4f} °C")
        print("-" * 16)
            
    except Exception as e:
        print(f"\nERROR CRÍTICO AL CALCULAR FÓRMULA:\n{e}")

calcular_rmse_sin_ai()

----------------
RMSE: 13.5042 °C
----------------


Se obtiene un $RMSE$ de $13.5$ºC. La idea de este proyecto por lo tanto es, combinando modelos de aprendizaje automático y redes neuronales sustentadas con modelos físicos, obtener un valor de $RMSE$ inferior.